# Load CSVs (one-to-many) to Azure AI Search

In this Jupyter Notebook, we create and run steps to index a CSV file in which each row is an individual and independent record/document. Each row then becomes searchable in Azure AI Search. 
The reference documentation can be found at [Indexing blobs and files to produce multiple search documents](https://learn.microsoft.com/en-us/azure/search/search-howto-index-one-to-many-blobs).

By default, an indexer will treat the contents of a blob or file as a single search document. If you want a more granular representation in a search index, you can set `parsingMode` values to create multiple search documents from one blob or file.

We are going to be using the same private Blob Storage account but a different container that has abstracts of 90k Medical publications about COVID-19 published in 2020-2022. This file is a subset of a much bigger dataset (770k articles) called CORD19 [HERE](https://github.com/allenai/cord19)

In [4]:
import os
import json
import requests
from dotenv import load_dotenv
from azure.identity import ManagedIdentityCredential

load_dotenv("credentials.env")

# Name of the container in your Blob Storage Datasource ( in credentials.env)
BLOB_CONTAINER_NAME = "csv"

In [2]:
# Define the names for the data source, index and indexer
datasource_name = "srch-datasource-csv-yk"
skillset_name = "srch-skillset-csv-yk"
index_name = "srch-index-csv-yk"
indexer_name = "srch-indexer-csv-yk"

In [5]:
credential = ManagedIdentityCredential(client_id="d30cba06-04c1-4065-a91d-8b7ce3b07b78")

# Obtain an access token for the Azure Cognitive Search service
token = credential.get_token("https://search.azure.com/.default")

print(f"Access token: {token.token}")

# Construct the headers with the access token
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {token.token}'
    }

params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

Access token: eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ikg5bmo1QU9Tc3dNcGhnMVNGeDdqYVYtbEI5dyIsImtpZCI6Ikg5bmo1QU9Tc3dNcGhnMVNGeDdqYVYtbEI5dyJ9.eyJhdWQiOiJodHRwczovL3NlYXJjaC5henVyZS5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8xNmIzYzAxMy1kMzAwLTQ2OGQtYWM2NC03ZWRhMDgyMGI2ZDMvIiwiaWF0IjoxNzI2OTg5MDc0LCJuYmYiOjE3MjY5ODkwNzQsImV4cCI6MTcyNzA3NTc3NCwiYWlvIjoiazJCZ1lOQTJqR3AwWDljNGYrV05VdUhsdVhkbEFRPT0iLCJhcHBpZCI6ImQzMGNiYTA2LTA0YzEtNDA2NS1hOTFkLThiN2NlM2IwN2I3OCIsImFwcGlkYWNyIjoiMiIsImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzE2YjNjMDEzLWQzMDAtNDY4ZC1hYzY0LTdlZGEwODIwYjZkMy8iLCJpZHR5cCI6ImFwcCIsIm9pZCI6ImNlN2VhNTVkLWFjZjktNDEwNy1hODkxLWNhMDRmNGQ0MTdiMyIsInJoIjoiMC5BVVlBRThDekZnRFRqVWFzWkg3YUNDQzIwNENqRFloZW1KaEJnYm5nV3h6Rk1WanhBQUEuIiwic3ViIjoiY2U3ZWE1NWQtYWNmOS00MTA3LWE4OTEtY2EwNGY0ZDQxN2IzIiwidGlkIjoiMTZiM2MwMTMtZDMwMC00NjhkLWFjNjQtN2VkYTA4MjBiNmQzIiwidXRpIjoiQkpOc0M3NU1DVVNfVVkyVG8tRVNBQSIsInZlciI6IjEuMCIsInhtc19pZHJlbCI6IjcgMTgiLCJ4bXNfbWlyaWQiOiIvc3Vic2NyaXB0aW9ucy9mZTM4YzM3Ni1iN

## Create Data Source (Blob container with the CSV data file)

In [6]:
# Create a data source

datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities of one-to-many.",
    "type": "azureblob",
    "credentials": {
        "connectionString": os.environ['BLOB_CONNECTION_STRING']
    },
    "container": {
        "name": BLOB_CONTAINER_NAME
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


## Inspect CSV file so we can understand the column types before creating the Index

In [ ]:
#Read data from data store
# Create data store and connect csv container 
#Follow the last step in the SET UP document
import pandas as pd

metadata = pd.read_csv("azureml://subscriptions/fe38c376-b42a-4741-9e7c-f5d7c31e5873/resourcegroups/yelizkilinc-rg/workspaces/aml-prod/datastores/aibootcamp/paths/all_sources_metadata.csv")
print("No. of lines:",metadata.shape[0])
metadata.head()

In our private dataset we have place a smaller version of the original the metadata.csv file in the cord19 dataset. 
Let's see what the file looks like:

## Create the Index
In Azure AI Search, both blob indexers and file indexers support a delimitedText parsing mode for CSV files that treats each line in the CSV as a separate search document.

### **Important**:
As you can see below and from the prior Notebook, there are 6 mandatory fields in the schema: `id, title, name, location, chunk, chunkVector`. These fields must exist in any index that you create regardles of the datasource. Any additional fields are good to add so the engine can search relevant documents, however the mandatory fields must exist for all the code downstream work with no issues.

In [7]:
# Create an index
# Queries operate over the searchable fields and filterable fields in the index
index_payload = {
    "name": index_name,
    "vectorSearch": {
        "algorithms": [
            {
                "name": "myalgo",
                "kind": "hnsw"
            }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters":
                {
                    "resourceUri" : os.environ['AZURE_OPENAI_ENDPOINT'],
                    "apiKey" : os.environ['AZURE_OPENAI_API_KEY'],
                    "deploymentId" : os.environ['EMBEDDING_DEPLOYMENT_NAME'],
                    "modelName" : os.environ['EMBEDDING_DEPLOYMENT_NAME'],
                }
            }
        ],
        "profiles": [
            {
                "name": "myprofile",
                "algorithm": "myalgo",
                "vectorizer":"openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    },
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "analyzer": "keyword", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false","facetable": "false"},
        {"name": "ParentKey", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "title", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},   
        {"name": "chunk","type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 1536,   # IMPORTANT: Make sure these dimmensions match your embedding model name
            "vectorSearchProfile": "myprofile",
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }
    ]
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)


201
True


## Create Skillset - Text Splitter, Language Detection
We don't need to use the OCR and Merge skill, since we know for sure that this is only text parsing. We will only use the SplitSkill and the AzureOpenAIEmbeddingSkill.

In [8]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description": "e2e Skillset for RAG - One to Many",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",
            "maximumPageLength": 5000, # 5000 is default
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/abstract"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "chunks"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.AzureOpenAIEmbeddingSkill",
            "description": "Azure OpenAI Embedding Skill",
            "context": "/document/chunks/*",
            "resourceUri": os.environ['AZURE_OPENAI_ENDPOINT'],
            "apiKey": os.environ['AZURE_OPENAI_API_KEY'],
            "deploymentId": os.environ['EMBEDDING_DEPLOYMENT_NAME'],
            "modelName" : os.environ['EMBEDDING_DEPLOYMENT_NAME'],
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/chunks/*"
                }
            ],
            "outputs": [
                {
                    "name": "embedding",
                    "targetName": "vector"
                }
            ]
        }
    ],
    "indexProjections": {
        "selectors": [
            {
                "targetIndexName": index_name,
                "parentKeyFieldName": "ParentKey",
                "sourceContext": "/document/chunks/*",
                "mappings": [
                    {
                        "name": "title",
                        "source": "/document/title"
                    },
                    {
                        "name": "name",
                        "source": "/document/name"
                    },
                    {
                        "name": "location",
                        "source": "/document/location"
                    },
                    {
                        "name": "chunk",
                        "source": "/document/chunks/*"
                    },
                    {
                        "name": "chunkVector",
                        "source": "/document/chunks/*/vector"
                    }
                ]
            }
        ],
        "parameters": {
            "projectionMode": "skipIndexingParentDocuments"
        }
    },
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": os.environ['COG_SERVICES_NAME'],
        "key": os.environ['COG_SERVICES_KEY']
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


## Create and Run the Indexer - (runs the pipeline)
To create one-to-many indexers with CSV blobs, create or update an indexer definition with the delimitedText parsing mode

In [9]:
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "schedule" : { "interval" : "PT30M"}, # How often do you want to check for new content in the data source
    "fieldMappings": [
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "name"
        },
        {
          "sourceFieldName" : "url",
          "targetFieldName" : "location"
        }
    ],
    "outputFieldMappings":
    [],
    "parameters" : { 
        "configuration" : { 
            "dataToExtract": "contentAndMetadata",
            "parsingMode" : "delimitedText", 
            "firstLineContainsHeaders" : True,
            "delimitedTextDelimiter": ","
        } 
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


In [15]:
# Optionally, get indexer status to confirm that it's running
try:
    r = requests.get(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name +
                     "/status", headers=headers, params=params)
    # pprint(json.dumps(r.json(), indent=1))
    print(r.status_code)
    print("Status:",r.json().get('lastResult').get('status'))
    print("Items Processed:",r.json().get('lastResult').get('itemsProcessed'))
    print(r.ok)
    
except Exception as e:
    print("Wait a few seconds until the process starts and run this cell again.")

200
Status: inProgress
Items Processed: 4268
True


**When the indexer finishes running we will have all 90,000 rows indexed properly as separate documents in our Search Engine!.**

# Reference

- https://learn.microsoft.com/en-us/azure/search/search-howto-index-csv-blobs



# NEXT
Now that we have two separate indexes loaded with two different types of information, In the next notebook 3, we will do a Multi-Index query, sort the results based on the reranker semantic score of Azure AI Search, and then use OpenAI to understand the results and give the best answer possible